In [ ]:
import fitz  # PyMuPDF
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [ ]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [ ]:
exceptions = []

In [ ]:
# Building system prompt
def get_verse_system_prompt():
    system_prompt = "You are a spiritual student who classifies the versus of the BhagavadGita according to a given theme.\n"
    system_prompt += "Given a theme, you should pick a verse from any chapter and give it's location in the form of index chapter.verse_number (6.2)\n"
    system_prompt += "You should respond in JSON as in this example:\n"
    system_prompt += """
        {"title": "Chapter 3, Verse 21 (3.21)", "verse": "कर्मणा ह्यपि संसिद्धिम्‌
                                                  आस्थिता जनकादय:।
                                                  लोकसंग्रहमेवापि
                                                  सम्पश्यन्कर्तुमर्हसि॥"}
        """
    return system_prompt

In [ ]:
print(get_verse_system_prompt())

In [ ]:
# Define user prompt
def get_verse_user_prompt(theme):
    user_prompt = f'''
                    Here is the theme : {theme},
                    Please find a verse from BhagavadGita excluding {exceptions} for a given theme {theme}
                    '''#excluding those results which are already used
    
    user_prompt += "If the verse is not in the exceptions for a given theme and used for a different theme, you are free to suggest it for a different theme."
    return user_prompt

In [ ]:
print(get_verse_user_prompt('motivation'))

In [ ]:
#Call openAI to return versus
def get_verses(theme):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": get_verse_system_prompt()},
            {"role": "user", "content": get_verse_user_prompt(theme)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    result = json.loads(result)

    #Remember those results which are suggested now
    combination = (theme, result['title'])
    exceptions.append(combination)
    return result

In [ ]:
print(get_verses('motivation'))

In [ ]:
# So far we have fetched the new verses relevant to a given theme 
# Lets generate a script for producting youtube video

In [ ]:
#def function for system prompt
def get_script_system_prompt(tone, theme, format):
    sys_prompt = 'You are a script writer for a youtube spiritual channel\n'
    sys_prompt += 'You are given a verse like below: \n'
    sys_prompt += str(get_verses(theme))
    sys_prompt += '\n'
    sys_prompt += f'Give me an engaging script in a {tone} tone for a {format} format video for audience like youth seeking purpose, spiritual seekers, indians abroad, scholars and curious minds.'

    return sys_prompt

In [ ]:
print(get_script_system_prompt('Motivating','motivation','long'))

In [ ]:
# def function for user prompt
def get_script_user_prompt(format, theme):
    user_prompt = f'Given the verse, help me generate a detailed script suitable for {format} format video.\n'
    user_prompt += f'Please give me the complete verse, its meaning, a relevant story having a dilemma which the verse solves and the interpretation of the verse with respect to {theme}.\n'
    user_prompt += 'Let the script give cues about video editing, host actions.'
    user_prompt += 'given the below example, please follow the format:\n'
    user_prompt += """
    [Opening Scene - Soft Instrumental Music Playing]

    [Cut to Host in a serene setting, perhaps by a river or in a lush green garden.]

    Host: (Smiling at the camera) "Namaste, dear viewers! Welcome back to our channel, where we explore the depths of spirituality and seek to ignite the flame of wisdom within you. Today, we delve into a profound verse from the Bhagavad Gita that speaks to the very essence of life and identity."

    [Text On Screen: Chapter 2, Verse 13 (2.13)]

    Host: (With a sense of reverence) "Let’s first take a moment to recite this verse together. It goes like this:

    देहिनोऽस्मिन्न्यथा देहे कौमारं यौवनं जरा।
    तथादेहान्तरप्राप्तिर्धीरस्तत्र न मुह्यति॥

    Now, let’s understand the essence of this verse."

    [Cut to Graphic: Verse Translation with Key Concepts Highlighted]

    Host Voiceover: (Calm and engaging tone) "The meaning of this beautiful verse translates to: 'Just as the body undergoes changes from childhood to old age, similarly, the soul transitions from one body to another. The wise, who understand this, are never bewildered by these changes.'

    [Cut back to Host]

    Host: (Nodding, creating a connection)
    "So, why is this verse so important, especially for us as young seekers of purpose? It highlights a profound truth—that our identities are not confined by our physical forms or the stages of life we experience. Instead, we are eternal beings who are constantly evolving."

    [Scene Transition - Soft Music Playing]

    [Cut to a Story Animation - A young man named Arjun in a busy city]

    Host (Voiceover): "Let me share a relatable story. Meet Arjun. Like many of us, he was once full of dreams and aspirations. He excelling in school, pursuing a career in engineering. But as the years passed, he faced a crossroads. As the pressure mounted, he began to question his identity.

    (Visuals show Arjun overwhelmed by societal expectations, with people pushing him in different directions.)

    He felt distinct phases of life pulling at him: childhood dreams, youthful ambitions, and the looming responsibilities of adulthood. The changing seasons of his life left him confused and wondering if he had lost his true self."

    [Cut back to Host, empathetic tone]

    Host: "Have you ever felt like Arjun? It’s a dilemma we all face, especially in today's fast-paced world where expectations can cloud our true identity. But just like our verse suggests, we should recognize that these changes don’t define us. They are simply part of the journey."

    [Scene Transition - Calm Music Playing while Host meditates]

    Host: (Speaking gently) "Let’s take a moment to reflect. When we are sad, does that sadness define us? Or when we achieve success, do we become defined solely by that success? The answer isn't as straightforward as it seems. Here’s the catch: our essence is beyond these transient states. Like the body, our identities are fluid."

    [Cut to Visuals of Nature - flowing rivers, trees shedding leaves, etc.]

    Host Voiceover: "Imagine the endless cycle of nature—the changing seasons, the growth, the decay, and rebirth. Just like the leaves that drop to make way for new growth, our experiences contribute to our spiritual evolution."

    [Cut back to Host - Inviting and Warm Tone]

    Host: "Just as the wise who understand the transformation of the soul remain unshaken, we, too, can cultivate that wisdom to rise above the chaos of change. Recognize your true essence—beyond the body, the roles, the titles. Understand that your spirit is eternal."

    [Scene Transition - Soft Inspirational Music Begins]

    Host: (Passionately) "So how can we embody this truth in our daily lives? Here’s a small exercise: Each day, take a few moments to meditate on who you really are. Write down what aspects of your identity are tied to transient things. Challenge yourself—what happens when you peel these layers away?"

    [Cut to host with a pad, writing ideas]

    [Scene Transition - Editing Cues - Show engaging graphics of identity, layers of a person, etc.]

    Host Voiceover: "Each effort towards understanding and embracing our true self draws us closer to the realization that we are eternal souls, having a human experience. This is the wisdom that can empower you to stand tall against the adversities of life."

    [Cut back to Host]

    Host: (Concluding) "Thank you for joining me today in this exploration of Chapter 2, Verse 13 of the Bhagavad Gita. Remember, when you feel lost in the complexities of life, return to this teachings and remind yourself that you are not just a body; you are an eternal being on a magnificent journey.

    [Closing Scene - Uplifting Music Playing]

    Host: "Don’t forget to like, share, and subscribe if you found resonance in this message. And share your thoughts in the comments below. What did you find most challenging in your own journey of self-identity? Let’s connect and support each other in our spiritual quests. Until next time, stay enlightened, stay inspired!"

    [End Screen with Subscribe Button and Previous Video Suggestions]

    [End of Script]
    """
    
    return user_prompt

In [ ]:
print(get_script_user_prompt('long','motivation'))

In [ ]:
def create_script(tone, theme, format):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": get_script_system_prompt(tone,theme,format)},
            {"role": "user", "content": get_script_user_prompt(format,theme)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [ ]:
create_script('motivating','self-identity','long')